In [227]:
from __future__ import print_function
import sys
import re
import numpy as np

from numpy import dot
from numpy.linalg import norm
from pyspark import SparkContext
from operator import add
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


Dimention=20000
smallTrainFile='SmallTrainingData.txt'
TestFile='TestingData.txt'

In [228]:
def freqArray (listOfIndices, numberofwords):
    returnVal = np.zeros (Dimention)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    returnVal = np.divide(returnVal, numberofwords)
    return returnVal


In [229]:
sc = SparkContext(appName="LogisticRegression2")

d_corpus = sc.textFile(smallTrainFile)

numberOfDocs =d_corpus.count()

d_keyAndText = d_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
d_keyAndListOfWords = d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

allWords = d_keyAndListOfWords.flatMap(lambda x: ((i,1) for i in x[1]))

allCounts = allWords.reduceByKey(lambda x,y: x+y)

topWords = allCounts.top(Dimention, lambda x : x[1])

topWordsK = sc.parallelize(range(Dimention))
dictionary = topWordsK.map (lambda x : (topWords[x][0], x))

In [230]:
allWordsWithDocID = d_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

allDictionaryWords = dictionary.join (allWordsWithDocID)


justDocAndPos = allDictionaryWords.map (lambda x: (x[1][1], x[1][0]))

allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], freqArray(x[1],len(x[1]))))
zeroOrOne = allDocsAsNumpyArrays.map(lambda x: (x[0], np.clip (np.multiply (x[1], 9e9), 0, 1)))
dfArray = zeroOrOne.reduce(lambda x1, x2: ("", np.add(x1[1], x2[1])))[1]
multiplier = np.full(Dimention, numberOfDocs)
idfArray = np.log(np.divide(np.full(Dimention, numberOfDocs), dfArray))

allDocsAsNumpyArraysTFidf = allDocsAsNumpyArrays.map(lambda x: (x[0], np.multiply(x[1], idfArray)))

# X_i
allDocsAsNumpyArraysTFidf.take(3)

[('AU466',
  array([2.95839346e-05, 1.81015855e-05, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])),
 ('AU1356',
  array([2.41667689e-05, 1.28667866e-05, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])),
 ('19264299',
  array([2.21781209e-05, 1.09011103e-05, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))]

For the test set

In [231]:
def get_label(x):
    if 'AU' in x[0]:
        return 1
    else:
        return 0

In [216]:
def get_vector(x):
    return (np.multiply(x[0],x[1]),np.multiply(x[1],(np.e**x[2]/(1+np.e**x[2]))),
            x[0]*x[2],np.log(np.e**float(x[2])+1))

In [242]:
# Without Regularization
R_current1=np.random.normal(0.1,0.05,Dimention)
learningRate=0.1
num_iteration = 100000
oldCost=0

t_allDocsAsNumpyArraysTFidf.cache()

for i in range(num_iteration):
   
    Y_X_Theta=t_allDocsAsNumpyArraysTFidf.map(lambda x: ((get_label(x), x[1],np.dot(x[1], R_current1))))
    Gradient=Y_X_Theta.map(get_vector)
    GradientSum=Gradient.reduce(lambda x,y: ((x[0]+y[0]),(x[1]+y[1]),(x[2]+y[2]),(x[3]+y[3])))
    
    # calculate cost:
    cost = GradientSum[3]-GradientSum[2]
    
    oldp = np.sqrt(np.square(R_current1).sum())
       
    R_current1= R_current1-learningRate*(-GradientSum[0]+GradientSum[1])
    
    p = np.sqrt(np.square(R_current1).sum())

    #Stop if the cost is not descreasing 
    if abs(oldp-p) <= 0.001:
        break

    if(cost<oldCost):
        learningRate=learningRate*1.05
        oldCost = cost

    if(cost>oldCost):
        learningRate=learningRate*0.5
        oldCost = cost
        

    print("Iteration No.=", i , " Cost=", cost) 
    print('R current:',R_current1)
  
    
   




Iteration No.= 0  Cost= 14418.465568890875
R current: [0.07613587 0.08972902 0.09937619 ... 0.13289558 0.03063214 0.07240296]
Iteration No.= 1  Cost= 11225.717823976443
R current: [0.07441968 0.08972902 0.09571098 ... 0.1281806  0.02696508 0.06799883]
Iteration No.= 2  Cost= 10080.482333035867
R current: [0.07276687 0.08972902 0.09217926 ... 0.1236139  0.0234144  0.06378615]
Iteration No.= 3  Cost= 9072.852388340616
R current: [0.07117808 0.08972902 0.0887821  ... 0.11919765 0.01998185 0.0597643 ]
Iteration No.= 4  Cost= 8188.8438078176105
R current: [0.06965254 0.08972902 0.08551754 ... 0.11493042 0.01666636 0.05592802]
Iteration No.= 5  Cost= 7414.288110271944
R current: [0.06818844 0.08972902 0.08238141 ... 0.11080806 0.01346473 0.05226883]
Iteration No.= 6  Cost= 6735.6041716758955
R current: [0.06678331 0.08972902 0.07936806 ... 0.10682451 0.01037225 0.0487763 ]
Iteration No.= 7  Cost= 6140.266482801507
R current: [0.06543425 0.08972902 0.07647096 ... 0.10297252 0.00738322 0.04543

Iteration No.= 63  Cost= 362.8767698537339
R current: [ 0.02483866  0.08972902 -0.01903693 ... -0.04305919 -0.11920362
 -0.06160426]
Iteration No.= 64  Cost= 348.9754926793728
R current: [ 0.02430138  0.08972902 -0.02043332 ... -0.04538266 -0.12197457
 -0.06311133]
Iteration No.= 65  Cost= 335.6559558109734
R current: [ 0.02376427  0.08972902 -0.02183006 ... -0.04770414 -0.12479594
 -0.06461373]
Iteration No.= 66  Cost= 322.8911985095241
R current: [ 0.02322721  0.08972902 -0.02322725 ... -0.05002353 -0.12766957
 -0.06611153]
Iteration No.= 67  Cost= 310.65546920018585
R current: [ 0.02269013  0.08972902 -0.02462499 ... -0.05234072 -0.13059728
 -0.06760479]
Iteration No.= 68  Cost= 298.92417505188314
R current: [ 0.02215294  0.08972902 -0.02602333 ... -0.05465562 -0.13358096
 -0.06909358]
Iteration No.= 69  Cost= 287.67383504203144
R current: [ 0.0216156   0.08972902 -0.02742235 ... -0.05696811 -0.13662252
 -0.07057796]
Iteration No.= 70  Cost= 276.88203597910444
R current: [ 0.0210780

Iteration No.= 125  Cost= 31.469935329585496
R current: [-0.00887239  0.08972902 -0.10643729 ... -0.18058781 -0.46494863
 -0.14968762]
Iteration No.= 126  Cost= 30.08663119415087
R current: [-0.00942493  0.08972902 -0.10787746 ... -0.18268353 -0.47461195
 -0.15110002]
Iteration No.= 127  Cost= 28.756169857263558
R current: [-0.00997895  0.08972902 -0.10932153 ... -0.18477499 -0.48442115
 -0.1525144 ]
Iteration No.= 128  Cost= 27.476985388148933
R current: [-0.0105346   0.08972902 -0.1107698  ... -0.1868621  -0.49437465
 -0.15393075]
Iteration No.= 129  Cost= 26.247587811948506
R current: [-0.01109207  0.08972902 -0.11222256 ... -0.1889447  -0.50447063
 -0.15534904]
Iteration No.= 130  Cost= 25.066557793662923
R current: [-0.01165152  0.08972902 -0.1136801  ... -0.19102266 -0.51470704
 -0.15676922]
Iteration No.= 131  Cost= 23.93253907495182
R current: [-0.01221313  0.08972902 -0.11514272 ... -0.19309579 -0.52508163
 -0.15819124]
Iteration No.= 132  Cost= 22.84422882742092
R current: [-

KeyboardInterrupt: 

In [232]:
def get_vector_regularization(x):
     return (np.multiply(x[0],x[1]),np.multiply(x[1],(np.e**float(x[2])/(1+np.e**float(x[2])))),
             x[0]*x[2],np.log(np.e**float(x[2])+1))
    

In [247]:
# With Regularization




R_current2 = np.random.normal(0, 0.1, Dimention)
learningRate = 0.1
num_iteration = 9999999
oldCost = 0

# I have tested many lambda, lambda=0.00001 can get a good result
lamda = 0.0001

allDocsAsNumpyArraysTFidf.cache()

for i in range(num_iteration):

    Y_X_Theta = allDocsAsNumpyArraysTFidf.map(lambda x: ((get_label(x), x[1], np.dot(x[1], R_current2))))
    Gradient = Y_X_Theta.map(get_vecor_regularization)
    GradientSum = Gradient.reduce(lambda x, y: ((x[0] + y[0]), (x[1] + y[1]), (x[2] + y[2]), (x[3] + y[3])))

    # calculate cost:
    cost = GradientSum[3] - GradientSum[2] + lamda * (np.square(R_current2).sum())

    oldp = np.sqrt(np.square(R_current2).sum())

    R_current2 = R_current2 - learningRate * (-GradientSum[0] + GradientSum[1])

    p = np.sqrt(np.square(R_current2).sum())

    # Stop if the cost is not descreasing
    if abs(oldp - p) <= 0.0001:
        break

    if (cost <= oldCost):
        learningRate = learningRate * 1.05
        oldCost = cost

    if (cost > oldCost):
        learningRate = learningRate * 0.5
        oldCost = cost

    print("Iteration No.=", i, " Cost=", cost)
    print('R current:', R_current2)




Iteration No.= 0  Cost= 2389.151163083214
R current: [ 0.08147695 -0.13810182  0.0837586  ...  0.0021481   0.02426707
 -0.06770935]
Iteration No.= 1  Cost= 2289.3961955614673
R current: [ 0.07965974 -0.13895048  0.0837586  ... -0.00154772  0.02111047
 -0.06925325]
Iteration No.= 2  Cost= 2242.288232748179
R current: [ 0.07778135 -0.13982781  0.0837586  ... -0.00536048  0.01784775
 -0.07085335]
Iteration No.= 3  Cost= 2194.411628458204
R current: [ 0.07584121 -0.14073409  0.0837586  ... -0.00929043  0.01447796
 -0.07251069]
Iteration No.= 4  Cost= 2145.827836430987
R current: [ 0.07383885 -0.14166956  0.0837586  ... -0.01333747  0.01100035
 -0.07422623]
Iteration No.= 5  Cost= 2096.604014093554
R current: [ 0.07177397 -0.14263436  0.0837586  ... -0.01750114  0.00741446
 -0.07600087]
Iteration No.= 6  Cost= 2046.812855354818
R current: [ 0.0696464  -0.14362859  0.0837586  ... -0.02178061  0.00372008
 -0.07783541]
Iteration No.= 7  Cost= 1996.532327110487
R current: [ 6.74561522e-02 -1.44

Iteration No.= 62  Cost= 247.62218000665405
R current: [-0.10122242 -0.22398664  0.0837586  ... -0.32927868 -0.29447235
 -0.26041495]
Iteration No.= 63  Cost= 237.8652659466306
R current: [-0.10424372 -0.2254115   0.0837586  ... -0.33434326 -0.29994435
 -0.26455081]
Iteration No.= 64  Cost= 228.48811257227243
R current: [-0.1072507  -0.22682932  0.0837586  ... -0.33937832 -0.30539915
 -0.26870325]
Iteration No.= 65  Cost= 219.47543461502272
R current: [-0.11024403 -0.22824038  0.0837586  ... -0.34438434 -0.31083679
 -0.27287194]
Iteration No.= 66  Cost= 210.81289263594934
R current: [-0.11322439 -0.22964499  0.0837586  ... -0.34936177 -0.31625728
 -0.27705653]
Iteration No.= 67  Cost= 202.48701652514376
R current: [-0.11619255 -0.23104348  0.0837586  ... -0.35431104 -0.32166065
 -0.2812567 ]
Iteration No.= 68  Cost= 194.4851309705415
R current: [-0.11914928 -0.23243621  0.0837586  ... -0.35923258 -0.32704691
 -0.2854721 ]
Iteration No.= 69  Cost= 186.7952835662827
R current: [-0.122095

Iteration No.= 124  Cost= 29.34114942468718
R current: [-0.2843638  -0.3092924   0.0837586  ... -0.59368614 -0.59717409
 -0.53409883]
Iteration No.= 125  Cost= 28.858051938743174
R current: [-0.28743955 -0.31069538  0.0837586  ... -0.59713875 -0.60135292
 -0.53858215]
Iteration No.= 126  Cost= 28.411419602063518
R current: [-0.2905223  -0.31210062  0.0837586  ... -0.60056614 -0.60550766
 -0.54306249]
Iteration No.= 127  Cost= 27.999992477009492
R current: [-0.29361218 -0.31350828  0.0837586  ... -0.6039684  -0.60963832
 -0.54753964]
Iteration No.= 128  Cost= 27.622550023121647
R current: [-0.29670931 -0.31491847  0.0837586  ... -0.60734558 -0.61374486
 -0.55201341]
Iteration No.= 129  Cost= 27.277908458649648
R current: [-0.29981377 -0.31633135  0.0837586  ... -0.61069777 -0.61782727
 -0.55648359]
Iteration No.= 130  Cost= 26.96491834569833
R current: [-0.30292564 -0.31774701  0.0837586  ... -0.61402505 -0.62188554
 -0.56094997]
Iteration No.= 131  Cost= 26.68246244844825
R current: [-

In [234]:
Index=R_current2.argsort()[::-1][:5]
Top5Words=[]
for i in range(5):
    Top5Words.append(dictionary.filter(lambda x:(x[1]==Index[i])).collect())
print(Top5Words)

[[('mr', 206)], [('applicant', 346)], [('tribunal', 485)], [('respondent', 709)], [('pty', 2320)]]


In [237]:
test_corpus = sc.textFile(TestFile)

numberOfDocs =test_corpus.count()

test_keyAndText = test_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
test_keyAndListOfWords = test_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

t_allWordsWithDocID = test_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

t_allDictionaryWords = dictionary.join (t_allWordsWithDocID)

t_justDocAndPos = t_allDictionaryWords.map (lambda x: (x[1][1], x[1][0]))

t_allDictionaryWordsInEachDoc = t_justDocAndPos.groupByKey()


t_allDocsAsNumpyArrays = t_allDictionaryWordsInEachDoc.map(lambda x: (x[0], freqArray(x[1],len(x[1]))))

t_zeroOrOne = t_allDocsAsNumpyArrays.map(lambda x: (x[0], np.clip (np.multiply (x[1], 9e9), 0, 1)))
t_dfArray = t_zeroOrOne.reduce(lambda x1, x2: ("", np.add(x1[1], x2[1])))[1]
t_idfArray = np.log(np.divide(np.full(20000, numberOfDocs), 1+t_dfArray))
t_allDocsAsNumpyArraysTFidf = t_allDocsAsNumpyArrays.map(lambda x: (x[0], np.multiply(x[1], t_idfArray)))

t_allDocsAsNumpyArraysTFidf .take(3)

[('13069936',
  array([2.58777583e-06, 0.00000000e+00, 8.62684098e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])),
 ('20591442',
  array([5.04650247e-06, 0.00000000e+00, 7.88600252e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])),
 ('20834540',
  array([5.31776108e-06, 0.00000000e+00, 6.64791150e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))]

In [243]:
def get_y_pred(x):
    if x[1]>0:
        return 1
    else:
        return 0

In [244]:
y_ypred1=t_allDocsAsNumpyArraysTFidf.map(lambda x: (get_label(x), np.dot(x[1], R_current1))).map(
                                                                lambda x: (x[0],get_y_pred(x)))
y1=y_ypred1.map(lambda x: x[0]).collect()
ypred1=y_ypred1.map(lambda x: x[1]).collect()

In [248]:
y_ypred2=t_allDocsAsNumpyArraysTFidf.map(lambda x: (get_label(x), np.dot(x[1], R_current2))).map(
                                                                lambda x: (x[0],get_y_pred(x)))
y2=y_ypred2.map(lambda x: x[0]).collect()
ypred2=y_ypred2.map(lambda x: x[1]).collect()

In [249]:
f1_score(y2, ypred2, average='binary')

0.9295774647887323

In [246]:
f1_score(y1, ypred1, average='binary')

1.0

In [250]:
confusion_matrix(y2, ypred2)

array([[18344,     3],
       [   47,   330]])

In [251]:
index = -1
FP_tab = []
for i,j in zip(y2,ypred2):
    index+=1
    if i==0 and j==1:
        FP_tab.append(index)
print(FP_tab)

[7928, 9568, 17213]


In [253]:
docIdList=t_allDocsAsNumpyArraysTFidf.map(lambda x:x[0]).collect()
print(docIdList[FP_tab[0]],docIdList[FP_tab[1]],docIdList[FP_tab[2]])

13442178 35810670 14683768


In [226]:
sc.stop()

In [ ]:
a=[1,2,3,4,5]
b=[1,2,3]
a[b[:3]]